In [1]:
import os 
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
SCORE_DIR = "data/scores"

In [3]:
box_scores = os.listdir(SCORE_DIR)

In [4]:
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if (f.endswith(".html"))] 
print(len(box_scores)) # Length is 8886 before removing 3 empty box scores
box_scores = [b for b in box_scores if os.path.getsize(b) != 0] # Remove empty box scores
print(len(box_scores)) # Length is 8883 after removing 3 empty box scores

8886
8883


In [5]:
def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()
        
    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup

In [6]:
def read_line_score(soup):
    line_score = pd.read_html(str(soup), attrs={"id": "line_score"})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols
    
    line_score = line_score[["team", "total"]]
    return line_score

In [7]:
def read_stats(soup, team, stat):
    df = pd.read_html(str(soup), attrs={"id": f"box-{team}-game-{stat}"}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df

In [8]:
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all("a")]
    season = os.path.basename(hrefs[1]).split("_")[0]
    
    return season

In [9]:
base_cols = None
games = []

for box_score in box_scores:
    soup = parse_html(box_score)
    line_score = read_line_score(soup)
    teams = list(line_score["team"])

    summaries = []

    for team in teams:
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")

        totals = pd.concat([basic.iloc[-1, :], advanced.iloc[-1, :]])
        totals.index = totals.index.str.lower()

        maxes = pd.concat([basic.iloc[:-1,:].max(), advanced.iloc[:-1,:].max()])
        maxes.index = maxes.index.str.lower() + "_max"

        summary = pd.concat([totals, maxes])

        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep="first"))
            base_cols = [b for b in base_cols if "bpm" not in b]

        summary = summary[base_cols]

        summaries.append(summary)

    summary = pd.concat(summaries, axis=1).T

    game = pd.concat([summary, line_score], axis=1)

    game["home"] = [0, 1]
    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"

    full_game = pd.concat([game, game_opp], axis=1)

    full_game["season"] = read_season_info(soup)

    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")

    full_game["win"] = full_game["total"] > full_game["total_opp"]
    games.append(full_game)
    
    print(f"{len(games)} / {len(box_scores)}")

1 / 8883
2 / 8883
3 / 8883
4 / 8883
5 / 8883
6 / 8883
7 / 8883
8 / 8883
9 / 8883
10 / 8883
11 / 8883
12 / 8883
13 / 8883
14 / 8883
15 / 8883
16 / 8883
17 / 8883
18 / 8883
19 / 8883
20 / 8883
21 / 8883
22 / 8883
23 / 8883
24 / 8883
25 / 8883
26 / 8883
27 / 8883
28 / 8883
29 / 8883
30 / 8883
31 / 8883
32 / 8883
33 / 8883
34 / 8883
35 / 8883
36 / 8883
37 / 8883
38 / 8883
39 / 8883
40 / 8883
41 / 8883
42 / 8883
43 / 8883
44 / 8883
45 / 8883
46 / 8883
47 / 8883
48 / 8883
49 / 8883
50 / 8883
51 / 8883
52 / 8883
53 / 8883
54 / 8883
55 / 8883
56 / 8883
57 / 8883
58 / 8883
59 / 8883
60 / 8883
61 / 8883
62 / 8883
63 / 8883
64 / 8883
65 / 8883
66 / 8883
67 / 8883
68 / 8883
69 / 8883
70 / 8883
71 / 8883
72 / 8883
73 / 8883
74 / 8883
75 / 8883
76 / 8883
77 / 8883
78 / 8883
79 / 8883
80 / 8883
81 / 8883
82 / 8883
83 / 8883
84 / 8883
85 / 8883
86 / 8883
87 / 8883
88 / 8883
89 / 8883
90 / 8883
91 / 8883
92 / 8883
93 / 8883
94 / 8883
95 / 8883
96 / 8883
97 / 8883
98 / 8883
99 / 8883
100 / 8883
101 / 88

In [12]:
games_df = pd.concat(games, ignore_index=True)

In [13]:
games_df.to_csv("nba_games.csv")